## Notebook setup


In [ ]:
import warnings
import datetime
import numpy as np
import pandas as pd

from IPython.display import HTML

from openbb_terminal import config_terminal as cfg
from openbb_terminal.helper_classes import TerminalStyle
from openbb_terminal import OpenBBFigure
from openbb_terminal.core.plots.plotly_ta.ta_class import PlotlyTA

warnings.filterwarnings("ignore")

In [ ]:
# Suppressing sdk logs
from openbb_terminal.core.session.current_system import set_system_variable

set_system_variable("LOGGING_SUPPRESS", True)

# Import the OpenBB SDK
from openbb_terminal.sdk import openbb, widgets

In [ ]:
from openbb_terminal.core.session.current_user import get_current_user

user = get_current_user()

from openbb_terminal import theme

cfg.theme = TerminalStyle(
    user.preferences.MPL_STYLE,
    user.preferences.PMF_STYLE,
    user.preferences.RICH_STYLE,
)
theme.apply_style("light")

In [ ]:
stylesheet = widgets.html_report_stylesheet()

## Select symbol


In [ ]:
# Parameters that will be replaced when calling this notebook
# Do not leave parameters blank as notebook will not run otherwise
symbol = "MSFT"
report_name = f"Equity report for {symbol}"

In [ ]:
if "." in symbol:
    import sys

    sys.exit(0)
symbol = symbol.upper()

In [ ]:
ticker_data = openbb.stocks.load(
    symbol=symbol, start_date=datetime.datetime.now() - datetime.timedelta(days=18 * 30)
)
ticker_data = openbb.stocks.process_candle(data=ticker_data)

author = ""
report_title = f"Investment Research Report on {symbol.upper()}"
report_date = datetime.datetime.now().strftime("%d %B, %Y")
report_time = datetime.datetime.now().strftime("%H:%M")
report_timezone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
report_title, report_date, report_time, report_timezone

In [ ]:
info = openbb.stocks.fa.overview(symbol=symbol, source="Finviz").transpose()

info
try:
    company_name = info["Company"][0]
except:
    company_name = "N/A"
try:
    company_sector = info["Sector"][0]
except:
    company_sector = "N/A"

try:
    company_industry = info["Industry"][0]
except:
    company_industry = "N/A"


overview = [company_name, company_sector, company_industry]

## Data


In [ ]:
try:
    (
        df_year_estimates,
        df_quarter_earnings,
        df_quarter_revenues,
    ) = openbb.stocks.fa.est(symbol=symbol)
    df_quarter_revenues.transpose()
except Exception as e:
    print(e)
    df_year_estimates = pd.DataFrame()
    df_quarter_earnings = pd.DataFrame()
    df_quarter_revenues = pd.DataFrame()

In [ ]:
display_year = sorted(df_year_estimates.columns.tolist())[:3]
df_year_estimates = df_year_estimates[display_year].head(5)

In [ ]:
tables = openbb.stocks.news(symbol)

for table in tables:
    if not isinstance(table, pd.DataFrame):
        continue

    table[0].loc["URL"] = table[0].loc["URL"].apply(lambda x: f'<a href="{x}">{x}</a>')

In [ ]:
quote_data = info[["Price", "Prev Close", "Change", "Volume", "52W Range", "Perf YTD"]]
quote_data

In [ ]:
try:
	df_institutional_shareholders = openbb.stocks.fa.shrs(symbol, holder="institutional")
	df_institutional_shareholders.index += 1
	df_institutional_shareholders
except Exception as e:
	print(e)
	df_institutional_shareholders = pd.DataFrame()

In [ ]:
# this command works 1/2 the time - really need to fix this - but this will catch the errors and stop it from breaking the report
try:
    df_sec_filings = openbb.stocks.fa.sec(symbol)[
        ["Company Name", "Reporting Owner", "Form Type", "Period", "View"]
    ].head(5)
    df_sec_filings
except:
    df_sec_filings = "Error with Nasdaq API"

In [ ]:
df_analyst = openbb.stocks.fa.pt(symbol=symbol).sort_index(ascending=False)
# print(df_analyst)
# if not df_analyst.empty:
#    if "target" in df_analyst.columns:
#        if "target_to" in df_analyst.columns:
#            df_analyst["target"] = df_analyst["target_to"].combine_first(
#                df_analyst["target"]
#            )
#        else:
#            df_analyst["target"] = df_analyst["target"]
#        df_analyst = df_analyst[["category", "analyst", "rating", "target"]].rename(
#            columns={
#                "category": "Category",
#                "analyst": "Analyst",
#                "rating": "Rating",
#                "target": "Price Target",
#            }
#        )
df_analyst.head(10)

In [ ]:
try:
	df_rating = openbb.stocks.fa.rating(symbol)
	df_rating
except Exception as e:
	print(e)
	df_rating = pd.DataFrame()

In [ ]:
pcr_chart = openbb.stocks.options.pcr_chart(symbol, window=30, external_axes=True)

if pcr_chart:
    pcr_chart = pcr_chart.to_html()

In [ ]:
current_price = 0
try:
    current_price = float(ticker_data["Adj Close"][-1])
    expiry_dates = openbb.stocks.options.expirations(symbol)

    if expiry_dates:
        exp = expiry_dates[0]
        option_chain = openbb.stocks.options.chains(symbol, expiration=exp)
except Exception as e:
    print(e)
    expiry_dates = []
    option_chain = pd.DataFrame()


In [ ]:
if expiry_dates:
    vol_chart = openbb.stocks.options.vol(
        option_chain, current_price, symbol, exp, external_axes=True
    )

    if vol_chart:
        vol_chart = vol_chart.to_html()

    voi_chart = openbb.stocks.options.voi(
        option_chain, current_price, symbol, exp, external_axes=True
    )

    if voi_chart:
        voi_chart = voi_chart.to_html()

In [ ]:
net_short_position = openbb.stocks.dps.spos_chart(
    symbol=symbol,
    limit=84,
    raw=False,
    export="",
    external_axes=True,
)

if net_short_position:
    net_short_position = net_short_position.to_html()

In [ ]:
dark_pools = openbb.stocks.dps.dpotc_chart(symbol=symbol, external_axes=True)

if dark_pools:
    dark_pools = dark_pools.to_html()

In [ ]:
gtrades_chart = openbb.stocks.gov.gtrades_chart(
    symbol,
    past_transactions_months=12,
    external_axes=True,
)

if gtrades_chart:
    gtrades_chart = gtrades_chart.to_html()

In [ ]:
gov_contracts_chart = openbb.stocks.gov.contracts_chart(
    symbol,
    past_transaction_days=365,
    external_axes=True,
)

if gov_contracts_chart:
    gov_contracts_chart = gov_contracts_chart.to_html()

In [ ]:
google_mentions_chart = openbb.stocks.ba.mentions_chart(
    symbol,
    external_axes=True,
)

if google_mentions_chart:
    google_mentions_chart = google_mentions_chart.to_html()

In [ ]:
google_regions_chart = openbb.stocks.ba.regions_chart(
    symbol,
    limit=10,
    external_axes=True,
)

if google_regions_chart:
    google_regions_chart = google_regions_chart.to_html()

In [ ]:
similar_companies = openbb.stocks.ca.similar(symbol, source="Finviz")[0:5]
if similar_companies:
    similar_companies.append(symbol)

In [ ]:
historical_similar = openbb.stocks.ca.hist_chart(
    similar_companies,
    external_axes=True,
    normalize=False,
)

if historical_similar:
    historical_similar = historical_similar.to_html()

In [ ]:
hcorr_similar = openbb.stocks.ca.hcorr_chart(
    similar_companies,
    external_axes=True,
)

if hcorr_similar:
    hcorr_similar = hcorr_similar.to_html()

In [ ]:
vol_similar = openbb.stocks.ca.volume_chart(
    similar_companies,
    external_axes=True,
)

if vol_similar:
    vol_similar = vol_similar.to_html()

In [ ]:
scorr_similar = openbb.stocks.ca.scorr_chart(
    similar_companies,
    external_axes=True,
)

if scorr_similar:
    scorr_similar = scorr_similar.to_html()

In [ ]:
income_comparison = openbb.stocks.ca.income(similar_companies)
income_comparison

In [ ]:
try:
    df_related_queries = openbb.stocks.ba.queries(symbol)
    df_related_queries.index += 1
    df_related_queries.index
except:
    pass

In [ ]:
try:
    df_rising_queries = openbb.stocks.ba.rise(symbol)
    df_rising_queries.index += 1
    df_rising_queries
except:
    pass

In [ ]:
gov_histcont_chart = openbb.stocks.gov.histcont_chart(
    symbol,
    external_axes=True,
)

if gov_histcont_chart:
    gov_histcont_chart = gov_histcont_chart.to_html()

In [ ]:
df_lobbying = openbb.stocks.gov.lobbying(symbol, limit=5)
df_lobbying

In [ ]:
price_vs_short_interest = openbb.stocks.dps.psi_sg_chart(
    symbol=symbol, external_axes=True
)

if price_vs_short_interest:
    price_vs_short_interest = price_vs_short_interest.to_html()

In [ ]:
price_chart = openbb.stocks.candle(
    symbol=symbol,
    data=ticker_data,
    external_axes=True,
)

if price_chart:
    price_chart = price_chart.to_html()

In [ ]:
try:
    price_target_chart = openbb.stocks.fa.pt_chart(
        symbol=symbol,
        start_date="2022-01-01",
        data=ticker_data,
        limit=10,
        raw=False,
        external_axes=True,
    )

    if price_target_chart:
        price_target_chart = price_target_chart.to_html()
except Exception:
    price_target_chart = OpenBBFigure()

In [ ]:
df = df_analyst.copy()
avg_ratings_last_30_days = 0
days = 0
if not df.empty:
    df_ratings = df[datetime.datetime.now() - datetime.timedelta(days=days) :]
    while df_ratings.empty:
        days += 30
        df_ratings = df[datetime.datetime.now() - datetime.timedelta(days=days) :]

        if days > 100:
            break

    if not df_ratings.empty:
        avg_ratings_last_30_days = round(np.mean(df_ratings["Price Target"].values), 2)
    else:
        avg_ratings = 0

last_price = round(ticker_data["Close"][-1], 2)

In [ ]:
ratings_over_time_chart = openbb.stocks.fa.rot_chart(
    symbol=symbol,
    limit=10,
    raw=False,
    export="",
    external_axes=True,
)

if ratings_over_time_chart:
    ratings_over_time_chart = ratings_over_time_chart.to_html()

In [ ]:
ta_rsi = openbb.ta.rsi_chart(ticker_data["Close"], external_axes=True)

if ta_rsi:
    ta_rsi = ta_rsi.to_html()

In [ ]:
df = openbb.ta.rsi(ticker_data["Close"])
rsi_value = round(df.values[-1][0], 2)
rsi_value

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(
    np.array(range(len(ticker_data["Close"][-30:].index))).reshape(-1, 1),
    ticker_data["Close"][-30:].values,
)
regression_slope = round(model.coef_[0], 2)

In [ ]:
df_insider = pd.DataFrame.from_dict(openbb.stocks.ins.lins(symbol=symbol)).head(10)
if not df_insider.empty:
    df_insider["Val ($)"] = df_insider["Value ($)"].replace({",": ""}, regex=True)
    df_insider["Trade"] = df_insider.apply(
        lambda row: (-1 * float(row["Val ($)"]))
        if row["Transaction"] == "Sale"
        else (float(row["Val ($)"]) if row["Transaction"] == "Buy" else 0),
        axis=1,
    )
    last_10_insider_trading = round(sum(df_insider["Trade"]) / 1_000_000, 2)
    df_insider = df_insider.drop(columns=["Val ($)", "Trade"])
    df_insider

In [ ]:
finbrain_sentiment = openbb.stocks.ba.headlines_chart(symbol=symbol, external_axes=True)

if finbrain_sentiment:
    finbrain_sentiment = finbrain_sentiment.to_html()

In [ ]:
try:
    df_sentiment_finbrain = openbb.stocks.ca.sentiment(symbols=[symbol])
    finbrain_sentiment_val = float(df_sentiment_finbrain.values[-1][0])
except Exception as e:
    finbrain_sentiment_val = "N/A"

In [ ]:
(
    watchlist_count,
    n_cases,
    n_bull,
    n_bear,
) = openbb.stocks.ba.bullbear(symbol=symbol)
stocktwits_sentiment = f"Watchlist count: {watchlist_count}</br>"
if n_cases > 0:
    stocktwits_sentiment += f"\nLast {n_cases} sentiment messages:</br>"
    stocktwits_sentiment += f"Bullish {round(100*n_bull/n_cases, 2)}%</br>"
    stocktwits_sentiment += f"Bearish {round(100*n_bear/n_cases, 2)}%"
else:
    stocktwits_sentiment += "No messages found"
stocktwits_sentiment

In [ ]:
snews = openbb.stocks.ba.snews_chart(symbol, external_axes=True)

if snews:
    snews = snews.to_html()

In [ ]:
ticker_data_all = openbb.stocks.load(
    symbol=symbol,
    start_date=datetime.datetime.now() - datetime.timedelta(days=5 * 12 * 21),
    verbose=False,
)
ticker_data_all["Returns"] = ticker_data_all["Adj Close"].pct_change()

In [ ]:
bw_month = openbb.qa.bw(
    ticker_data_all,
    "Returns",
    symbol,
    yearly=False,
    external_axes=True,
)

if bw_month:
    bw_month = bw_month.to_html()

In [ ]:
bw_year = openbb.qa.bw(
    ticker_data_all,
    "Returns",
    symbol,
    yearly=True,
    external_axes=True,
)

if bw_year:
    bw_year = bw_year.to_html()

In [ ]:
income_df = openbb.stocks.fa.income(symbol, source="FinancialModelingPrep")
data_df = openbb.stocks.fa.overview(symbol, source="FinancialModelingPrep")
mgmt_df = openbb.stocks.fa.mgmt(symbol)
try:
    if not mgmt_df.empty:
        mgmt_df["Info"] = mgmt_df["Info"].apply(lambda x: f'<a href="{x}">{x}</a>')
        mgmt_df["Insider Activity"] = mgmt_df["Insider Activity"].apply(
            lambda x: f'<a href="{x}">{x}</a>' if x != "-" else x
        )
except:
    pass
try:
    hist_dcf = openbb.stocks.fa.dcfc(symbol)
except:
    pass

enterprise_df = openbb.stocks.fa.enterprise(symbol)
score = openbb.stocks.fa.score(symbol, years=10)

In [ ]:
indicators = dict(
    ema=dict(length=[50]),
    sma=dict(length=[50]),
    wma=dict(length=[50]),
)

ma_chart = PlotlyTA.plot(ticker_data, indicators, symbol, candles=False, volume=False)

if ma_chart:
    ma_chart = ma_chart.show(external=True)
    ma_chart = ma_chart.to_html()

In [ ]:
macd_chart = openbb.ta.macd_chart(
    ticker_data["Close"], symbol=symbol, external_axes=True
)

if macd_chart:
    macd_chart = macd_chart.to_html()

In [ ]:
cci_chart = openbb.ta.cci_chart(ticker_data, symbol=symbol, external_axes=True)

if cci_chart:
    cci_chart = cci_chart.to_html()

In [ ]:
stoch_chart = openbb.ta.stoch_chart(ticker_data, symbol=symbol, external_axes=True)

if stoch_chart:
    stoch_chart = stoch_chart.to_html()

In [ ]:
adx_chart = openbb.ta.adx_chart(ticker_data, symbol=symbol, external_axes=True)

if adx_chart:
    adx_chart = adx_chart.to_html()

In [ ]:
bbands_chart = openbb.ta.bbands_chart(ticker_data, symbol=symbol, external_axes=True)

if bbands_chart:
    bbands_chart = bbands_chart.to_html()

In [ ]:
ad_chart = openbb.ta.ad_chart(ticker_data, symbol=symbol, external_axes=True)

if ad_chart:
    ad_chart = ad_chart.to_html()

## Render the report template to a file


In [ ]:
body = ""


body += widgets.header(
    author,
    report_date,
    report_time,
    report_timezone,
    report_title,
    plotly_js=True,
)

body += widgets.tablinks(
    [
        "SUMMARY",
        "Overview",
        "Fundamental Analysis",
        "Technical Analysis",
        "Behavioural Analysis",
        "Government Menu",
        "Comparison Menu",
        "Options",
        "Dark Pool and Shorts",
        "Quantitative Analysis",
        "Analyst Opinions",
        "Insider Trading",
        #    "Prediction Techniques",
    ]
)

htmlcode = widgets.h(3, "KPIs")
htmlcode += widgets.kpi(
    [last_price],
    [
        "Last closing price is above the average price ratings of last 30 days",
        "Average price ratings of last 30 day is above last closing price",
    ],
    avg_ratings_last_30_days,
)
htmlcode += widgets.kpi(
    [30, 70],
    ["RSI level is oversold", "RSI level is normal", "RSI level is overbought"],
    rsi_value,
)
try:
    htmlcode += widgets.kpi(
        [0],
        [
            "The sum of last 10 insider trading (in millions) was negative",
            "The sum of last 10 insider trading (in millions) was positive",
        ],
        last_10_insider_trading,
    )
except:
    pass
try:
    htmlcode += widgets.kpi(
        [-0.1, 0.1],
        [
            "Last FinBrain sentiment is bearish",
            " Last FinBrain sentiment is neutral",
            "Last FinBrain sentiment is bullish",
        ],
        finbrain_sentiment_val,
    )
except:
    pass
if score and score["Total Score"]:
    htmlcode += widgets.kpi(
        [25, 75],
        [
            "Buffet Score is not favourable",
            "Buffet Score is neutral",
            "Buffet Score is favourable",
        ],
        float(score["Total Score"]),
    )
body += widgets.add_tab("SUMMARY", htmlcode)

htmlcode = ""
if overview:
    titles = ["Company", "Sector", "Industry"]
    m = 0
    for x in overview:
        print(x)
        htmlcode += widgets.row([widgets.h(3, titles[m]) + widgets.p(x)])
        m += 1
htmlcode += widgets.row([widgets.h(3, "Price Chart") + (price_chart or "")])

try:
    htmlcode += widgets.row([widgets.h(3, "Quote") + quote_data.to_html()])
except:
    pass

htmlcode += widgets.row([widgets.h(3, "Latest News for " + symbol)])
try:
    for table in tables:
        htmlcode += widgets.row(
            [widgets.h(4, table[1]["title"]) + table[0].to_html(escape=False)]
        )
except:
    pass

if not mgmt_df.empty:
    htmlcode += widgets.row(
        [widgets.h(3, f"Management team of {symbol}") + mgmt_df.to_html(escape=False)]
    )
body += widgets.add_tab("Overview", htmlcode)

htmlcode = widgets.row(
    [widgets.h(3, "Price Target Chart") + (price_target_chart or "")]
)
htmlcode += widgets.row(
    [widgets.h(3, "Analyst Ratings over time") + (ratings_over_time_chart or "")]
)
try:
    htmlcode += widgets.row([widgets.h(3, "Analyst Ratings") + df_analyst.to_html()])
except:
    pass
# try:
#    htmlcode += widgets.row(
#        [widgets.h(3, "Analyst Recommendations") + df_rating.to_html()]
#    )
# except:
#    pass

body += widgets.add_tab("Analyst Opinions", htmlcode)

try:
    htmlcode = widgets.row(
        [widgets.h(3, "Estimates") + df_year_estimates.head().to_html()]
    )
except:
    pass

if not df_quarter_earnings.empty:
    htmlcode += widgets.row(
        [widgets.h(3, "Earnings") + df_quarter_earnings.head().to_html()]
    )
if not df_quarter_revenues.empty:
    htmlcode += widgets.row(
        [widgets.h(3, "Revenues") + df_quarter_revenues.head().to_html()]
    )
try:
    htmlcode += widgets.row(
        [
            widgets.h(3, "Major Institutional Shareholders")
            + df_institutional_shareholders.head().to_html()
        ]
    )
except:
    pass
try:
    htmlcode += widgets.row(
        [widgets.h(3, f"Historical DCF for {symbol}") + hist_dcf.to_html()]
    )
except:
    pass
try:
    htmlcode += widgets.row(
        [widgets.h(3, f"Enterprise data for {symbol}") + enterprise_df.to_html()]
    )
except:
    pass
try:
    htmlcode += widgets.row(
        [widgets.h(3, f"Income Statement for {symbol}") + income_df.to_html()]
    )
except:
    pass
try:
    htmlcode += widgets.row([widgets.h(3, f"Data for {symbol}") + data_df.to_html()])
except:
    pass
try:
    htmlcode += widgets.row(
        [widgets.h(3, "SEC filings") + df_sec_filings.to_html(escape=False)]
    )
except:
    pass

body += widgets.add_tab("Fundamental Analysis", htmlcode)

htmlcode = widgets.row([widgets.h(3, "Put to call ratio") + (pcr_chart or "")])

if expiry_dates:
    htmlcode += widgets.row(
        [widgets.h(3, "Option Volume for closest expiry date") + (vol_chart or "")]
    )
    htmlcode += widgets.row(
        [
            widgets.h(3, "Volume and Open Interest for closest expiry date")
            + (voi_chart or "")
        ]
    )

body += widgets.add_tab("Options", htmlcode)

htmlcode = widgets.row([net_short_position or ""])
htmlcode += widgets.row([price_vs_short_interest or ""])
# htmlcode += widgets.row([dark_pools])
body += widgets.add_tab("Dark Pool and Shorts", htmlcode)

htmlcode = widgets.row(
    [
        widgets.h(3, "Congress trading in the past 12 months for " + symbol)
        + (gtrades_chart or "")
    ]
)
htmlcode += widgets.row([gov_histcont_chart or ""])
htmlcode += widgets.row([gov_contracts_chart or ""])
htmlcode += widgets.row([widgets.h(3, "Recent Corporate Lobbying by " + symbol)])
try:
    for _, row in df_lobbying.iterrows():
        amount = (
            "$" + str(int(float(row["Amount"]))) if row["Amount"] is not None else "N/A"
        )
        htmlcode += widgets.row([widgets.p(f"{row['Date']}: {row['Client']} {amount}")])
        if (row["Amount"] is not None) and (row["Specific_Issue"] is not None):
            htmlcode += widgets.row(
                [
                    widgets.p(
                        "\t"
                        + row["Specific_Issue"].replace("\n", " ").replace("\r", "")
                    )
                ]
            )
except:
    pass

body += widgets.add_tab("Government Menu", htmlcode)

htmlcode = widgets.row(
    [
        widgets.h(3, f"Price over the past year for companies similar to {symbol}")
        + (historical_similar or "")
    ]
)
htmlcode += widgets.row(
    [
        widgets.h(3, f"Price correlation with similar companies for {symbol}")
        + (hcorr_similar or "")
    ]
)
htmlcode += widgets.row(
    [
        widgets.h(3, f"Volume over the past year for companies similar to {symbol}")
        + (vol_similar or "")
    ]
)
htmlcode += widgets.row(
    [
        widgets.h(3, f"Sentiment correlation with similar companies for {symbol}")
        + (scorr_similar or "")
    ]
)
if isinstance(income_comparison, pd.DataFrame):
    htmlcode += widgets.row(
        [
            widgets.h(3, f"Income data for similar companies to {symbol}")
            + income_comparison.to_html()
        ]
    )
htmlcode += widgets.row(
    [
        widgets.p(
            "Note that similar companies have been found using openbb.stocks.ca.similar"
        )
    ]
)
body += widgets.add_tab("Comparison Menu", htmlcode)

htmlcode = widgets.row([widgets.h(3, f"Moving Averages for {symbol}") + ma_chart])
htmlcode += widgets.row([macd_chart or ""])
htmlcode += widgets.row([ta_rsi or ""])
htmlcode += widgets.row([stoch_chart or ""])
htmlcode += widgets.row([cci_chart or ""])
htmlcode += widgets.row([ad_chart or ""])
htmlcode += widgets.row([bbands_chart or ""])
htmlcode += widgets.row([adx_chart or ""])
body += widgets.add_tab("Technical Analysis", htmlcode)

if not df_insider.empty:
    htmlcode = widgets.row(
        [widgets.h(3, "Last Activity") + df_insider.head(10).to_html(col_space="75px")]
    )
else:
    htmlcode = widgets.row(
        [widgets.h(3, "Last Activity") + widgets.p("No insider trading data found.")]
    )

body += widgets.add_tab("Insider Trading", htmlcode)


htmlcode = widgets.row([bw_year or ""])
htmlcode += widgets.row([bw_month or ""])
body += widgets.add_tab("Quantitative Analysis", htmlcode)


htmlcode = widgets.row([finbrain_sentiment or ""])
htmlcode += widgets.row([snews or ""])
htmlcode += widgets.row(
    [
        widgets.h(3, "Interest in " + symbol + " based on google analytics")
        + (google_mentions_chart or "")
    ]
)
htmlcode += widgets.row(
    [
        widgets.h(3, f"Regions with highest interest in {symbol}")
        + (google_regions_chart or "")
    ]
)
try:
    htmlcode += widgets.row(
        [
            widgets.h(3, f"Top queries related to {symbol}")
            + df_related_queries.to_html()
        ]
    )
    htmlcode += widgets.row(
        [
            widgets.h(3, f"Top rising queries related to {symbol}")
            + df_rising_queries.to_html()
        ]
    )
except:
    pass
htmlcode += widgets.row(
    [widgets.h(3, f"Stocktwits sentiment for {symbol}") + stocktwits_sentiment]
)
body += widgets.add_tab("Behavioural Analysis", htmlcode)

body += widgets.tab_clickable_and_save_evt()

report = widgets.html_report(
    title=report_name, stylesheet=widgets.html_report_stylesheet(), body=body
)

# to save the results
with open(report_name + ".html", "w", encoding="utf-8") as fh:
    fh.write(report)